In [1]:
import transformers
import torch
from transformers import BertForSequenceClassification

root_path = "/home/mai.kassem/Documents/Med-BERT/"
BertForSequenceClassification.from_pretrained(root_path + "MedBERT_Pretraining/")

/home/mai.kassem/.conda/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
-

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(82603, 192, padding_idx=0)
      (position_embeddings): Embedding(512, 192)
      (token_type_embeddings): Embedding(1000, 192)
      (LayerNorm): LayerNorm((192,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=192, out_features=192, bias=True)
              (key): Linear(in_features=192, out_features=192, bias=True)
              (value): Linear(in_features=192, out_features=192, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=192, out_features=192, bias=True)
              (LayerNorm): LayerNorm((192,), eps=1e-12, elem

In [2]:
torch.load(root_path + "MedBERT_Pretraining/pytorch_model.bin")

OrderedDict([('bert.embeddings.position_ids',
              tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
                        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
                        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
                        42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
                        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
                        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
                        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
                        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
                       112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
                       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
                       1

In [3]:
# Required Packages
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from transformers import BertForSequenceClassification
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import matplotlib
import transformers
import time
import tqdm
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
from termcolor import colored
import math
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import numpy as np
from datetime import datetime
import pickle as pkl
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
%matplotlib inline
use_cuda = torch.cuda.is_available()


In [4]:
torch.cuda.is_available()

True

In [5]:
train_f = pkl.load(
    open(
        root_path + "data/finetuning_data_updated/finetuning_gentacimin_2.bencs.train",
        "rb",
    ),
    encoding="bytes",
)
valid_f = pkl.load(
    open(
        root_path + "data/finetuning_data_updated/finetuning_gentacimin_2.bencs.valid",
        "rb",
    ),
    encoding="bytes",
)
test_f = pkl.load(
    open(
        root_path + "data/finetuning_data_updated/finetuning_gentacimin_2.bencs.test",
        "rb",
    ),
    encoding="bytes",
)

In [6]:
print("Length of training dataset:", len(train_f))
print("length of validation dataset", len(valid_f))
print("length of testing dataset", len(test_f))

Length of training dataset: 12038
length of validation dataset 1719
length of testing dataset 3439


In [7]:
train_f[0]

[11046564,
 1,
 [8931, 261, 1031, 192, 4026, 205, 11799, 397, 2391, 3107],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [8]:
# The version in this file is updated to dump pt_id for better visualization and results analysis

# Below are key functions for  Data prepartion ,formating input data into features, and model defintion
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    Based on original BERT code: We use this class instead of `None` because treating `None` as padding
    batches could cause silent errors.
    """


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(
        self, input_ids, input_mask, segment_ids, label_id, pt_id, is_real_example=True
    ):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.is_real_example = is_real_example
        self.pt_id = pt_id


def convert_EHRexamples_to_features(examples, max_seq_length):
    """Convert a set of `InputExample`s to a list of `InputFeatures`."""

    features = []
    for (ex_index, example) in enumerate(examples):
        feature = convert_singleEHR_example(ex_index, example, max_seq_length)
        features.append(feature)
    return features


# This is the EHR version
def convert_singleEHR_example(ex_index, example, max_seq_length):
    if isinstance(example, PaddingInputExample):
        return InputFeatures(
            input_ids=[0] * max_seq_length,
            input_mask=[0] * max_seq_length,
            segment_ids=[0] * max_seq_length,
            label_id=0,
            is_real_example=False,
        )

    input_ids = example[2]
    segment_ids = example[3]
    label_id = example[1]
    pt_id = example[0]

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # LR 5/13 Left Truncate longer sequence
    while len(input_ids) > max_seq_length:
        input_ids = input_ids[-max_seq_length:]
        input_mask = input_mask[-max_seq_length:]
        segment_ids = segment_ids[-max_seq_length:]

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    feature = [input_ids, input_mask, segment_ids, label_id, pt_id, True]
    return feature

In [9]:
class BERTdataEHR(Dataset):
    def __init__(self, Features):

        self.data = Features

    def __getitem__(self, idx, seeDescription=False):

        sample = self.data[idx]

        return sample

    def __len__(self):
        return len(self.data)


# customized parts for EHRdataloader
def my_collate(batch):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []
    all_pt_ids = []

    for feature in batch:
        all_input_ids.append(feature[0])
        all_input_mask.append(feature[1])
        all_segment_ids.append(feature[2])
        all_label_ids.append(feature[3])
        all_pt_ids.append(feature[4])
    return [[all_input_ids, all_input_mask, all_segment_ids, all_label_ids], all_pt_ids]


class BERTdataEHRloader(DataLoader):
    def __init__(
        self,
        dataset,
        batch_size=128,
        shuffle=False,
        sampler=None,
        batch_sampler=None,
        num_workers=0,
        collate_fn=my_collate,
        pin_memory=False,
        drop_last=False,
        timeout=0,
        worker_init_fn=None,
    ):
        DataLoader.__init__(
            self,
            dataset,
            batch_size=batch_size,
            shuffle=False,
            sampler=None,
            batch_sampler=None,
            num_workers=0,
            collate_fn=my_collate,
            pin_memory=False,
            drop_last=False,
            timeout=0,
            worker_init_fn=None,
        )
        self.collate_fn = collate_fn

In [10]:
class EHR_BERT_LR(nn.Module):
    def __init__(
        self,
        input_size,
        embed_dim,
        hidden_size,
        n_layers=1,
        dropout_r=0.1,
        cell_type="LSTM",
        bi=False,
        time=False,
        preTrainEmb="",
    ):
        super(EHR_BERT_LR, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.dropout_r = dropout_r
        self.cell_type = cell_type
        self.preTrainEmb = preTrainEmb
        self.time = time

        if bi:
            self.bi = 2
        else:
            self.bi = 1

        self.PreBERTmodel = BertForSequenceClassification.from_pretrained(
            root_path + "MedBERT_Pretraining/"
        )
        if use_cuda:
            self.PreBERTmodel.cuda()
        input_size = self.PreBERTmodel.bert.config.vocab_size
        self.in_size = self.PreBERTmodel.bert.config.hidden_size

        self.dropout = nn.Dropout(p=self.dropout_r)
        self.out = nn.Linear(self.in_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()
        if use_cuda:
            self.flt_typ = torch.cuda.FloatTensor
            self.lnt_typ = torch.cuda.LongTensor
        else:
            self.lnt_typ = torch.LongTensor
            self.flt_typ = torch.FloatTensor

    def forward(self, sequence):
        token_t = torch.from_numpy(np.asarray(sequence[0][0], dtype=int)).type(
            self.lnt_typ
        )
        seg_t = torch.from_numpy(np.asarray(sequence[0][2], dtype=int)).type(
            self.lnt_typ
        )
        Label_t = torch.from_numpy(np.asarray(sequence[0][3], dtype=int)).type(
            self.lnt_typ
        )
        Bert_out = self.PreBERTmodel.bert(
            input_ids=token_t,
            attention_mask=torch.from_numpy(np.asarray(sequence[0][1], dtype=int)).type(
                self.lnt_typ
            ),
            token_type_ids=seg_t,
        )
        output = self.sigmoid(self.out(Bert_out[1]))
        return output.squeeze(), Label_t.type(self.flt_typ)

In [11]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def trainsample(sample, model, optimizer, criterion=nn.BCELoss()):
    model.train()
    model.zero_grad()
    # print(sample[0])
    output, label_tensor = model(sample)
    loss = criterion(output, label_tensor)
    loss.backward()
    optimizer.step()
    return output, loss.item()


# train with loaders


def trainbatches(mbs_list, model, optimizer, shuffle=True):
    current_loss = 0
    all_losses = []
    plot_every = 5
    n_iter = 0
    if shuffle:
        random.shuffle(mbs_list)
    for i, batch in enumerate(mbs_list):
        output, loss = trainsample(batch, model, optimizer, criterion=nn.BCELoss())
        current_loss += loss
        n_iter += 1

        if n_iter % plot_every == 0:
            all_losses.append(current_loss / plot_every)
            current_loss = 0
    return current_loss, all_losses


def calculate_auc(model, mbs_list, shuffle=True):
    model.eval()
    y_real = []
    y_hat = []
    if shuffle:
        random.shuffle(mbs_list)
    for i, batch in enumerate(mbs_list):
        output, label_tensor = model(batch)
        y_hat.extend(output.cpu().data.view(-1).numpy())
        y_real.extend(label_tensor.cpu().data.view(-1).numpy())
    auc = roc_auc_score(y_real, y_hat)
    return auc, y_real, y_hat


# define the final epochs running, use the different names


def epochs_run(
    epochs,
    train,
    valid,
    test,
    model,
    optimizer,
    shuffle=True,
    patience=20,
    output_dir="../models/",
    model_prefix="dhf.train",
    model_customed="",
):
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0
    # header = 'BestValidAUC|TestAUC|atEpoch'
    # logFile = output_dir + model_prefix + model_customed +'EHRmodel.log'
    # print2file(header, logFile)
    # writer = SummaryWriter(output_dir+'/tsb_runs/') ## LR added 9/27 for tensorboard integration
    for ep in range(epochs):
        print(ep)
        start = time.time()
        current_loss, train_loss = trainbatches(
            mbs_list=train, model=model, optimizer=optimizer
        )
        train_time = timeSince(start)
        # epoch_loss.append(train_loss)
        avg_loss = np.mean(train_loss)
        # writer.add_scalar('Loss/train', avg_loss, ep) ## LR added 9/27
        valid_start = time.time()
        train_auc, _, _ = calculate_auc(model=model, mbs_list=train, shuffle=shuffle)
        valid_auc, _, _ = calculate_auc(model=model, mbs_list=valid, shuffle=shuffle)
        valid_time = timeSince(valid_start)
        # writer.add_scalar('train_auc', train_auc, ep) ## LR added 9/27
        # writer.add_scalar('valid_auc', valid_auc, ep) ## LR added 9/27
        print(
            colored(
                "\n Epoch (%s): Train_auc (%s), Valid_auc (%s) ,Training Average_loss (%s), Train_time (%s), Eval_time (%s)"
                % (ep, train_auc, valid_auc, avg_loss, train_time, valid_time),
                "green",
            )
        )
        if valid_auc > bestValidAuc:
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            best_model = model
            bestTrainAuc = train_auc
            if test:
                testeval_start = time.time()
                bestTestAuc, _, _ = calculate_auc(
                    model=best_model, mbs_list=test, shuffle=shuffle
                )

                # writer.add_scalar('test_auc', valid_auc, ep) ## LR added 9/27
                print(
                    colored(
                        "\n Test_AUC (%s) , Test_eval_time (%s) "
                        % (bestTestAuc, timeSince(testeval_start)),
                        "yellow",
                    )
                )
                # print(best_model,model) ## to verify that the hyperparameters already impacting the model definition
                # print(optimizer)
        if ep - bestValidEpoch > patience:
            break

    # writer.close()
    # if not os.path.exists(output_dir):
    #    os.makedirs(output_dir)
    # save model & parameters
    # torch.save(best_model, output_dir + model_prefix + model_customed + 'EHRmodel.pth')
    # torch.save(best_model.state_dict(), output_dir + model_prefix + model_customed + 'EHRmodel.st')

    if test:
        print(
            colored(
                "BestValidAuc %f has a TestAuc of %f at epoch %d "
                % (bestValidAuc, bestTestAuc, bestValidEpoch),
                "green",
            )
        )
        return bestTrainAuc, bestValidAuc, bestTestAuc, bestValidEpoch
    else:
        print(
            colored(
                "BestValidAuc %f at epoch %d " % (bestValidAuc, bestValidEpoch), "green"
            )
        )
        print("No Test Accuracy")

    print(
        colored(
            "Details see ../models/%sEHRmodel.log" % (model_prefix + model_customed),
            "green",
        )
    )

In [12]:
MAX_SEQ_LENGTH = 75
BATCH_SIZE = 100
LEARNING_RATE = 1e-3
NUM_EPOCHS = 91

results = []

# Data Preparation
train_features = convert_EHRexamples_to_features(train_f, MAX_SEQ_LENGTH)
test_features = convert_EHRexamples_to_features(test_f, MAX_SEQ_LENGTH)
valid_features = convert_EHRexamples_to_features(valid_f, MAX_SEQ_LENGTH)
train = BERTdataEHR(train_features)
test = BERTdataEHR(test_features)
valid = BERTdataEHR(valid_features)
print(" creating the list of training minibatches")
train_mbs = list(BERTdataEHRloader(train, batch_size=BATCH_SIZE))
print(" creating the list of test minibatches")
test_mbs = list(BERTdataEHRloader(test, batch_size=BATCH_SIZE))
print(" creating the list of valid minibatches")
valid_mbs = list(BERTdataEHRloader(valid, batch_size=BATCH_SIZE))

for run in range(10):  # to average the results on 10 runs
    for model_type in ["Bert only"]:
        ehr_model = EHR_BERT_LR(input_size=90000, embed_dim=192, hidden_size=192)
        if use_cuda:
            ehr_model.cuda()
        optimizer = optim.Adam(
            ehr_model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5
        )
        out_dir_name = "Gentacimin_finetuning"  # + str(i)
        trauc, vauc, testauc, bep = epochs_run(
            NUM_EPOCHS,
            train=train_mbs,
            valid=valid_mbs,
            test=test_mbs,
            model=ehr_model,
            optimizer=optimizer,
            shuffle=True,
            # batch_size = args.batch_size,
            patience=20,
            output_dir=out_dir_name,
            model_prefix="first_run",
        )
        results.append(
            [
                model_type,
                run,
                len(train_features),
                len(test_features),
                len(valid_features),
                trauc,
                vauc,
                testauc,
                bep,
            ]
        )

 creating the list of training minibatches
 creating the list of test minibatches
 creating the list of valid minibatches


Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForS

0

 Epoch (0): Train_auc (0.7458129575797803), Valid_auc (0.5654629861982434) ,Training Average_loss (0.2815218127643069), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6106615366261036) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.7682603571471412), Valid_auc (0.6141706398996236) ,Training Average_loss (0.258651839569211), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.5445776664280602) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.5383837212556928), Valid_auc (0.5084240903387703) ,Training Average_loss (0.2674365228662888), Train_time (0m 5s), Eval_time (0m 1s)
3

 Epoch (3): Train_auc (0.5818002702994486), Valid_auc (0.5262308657465495) ,Training Average_loss (0.26642555234332876), Train_time (0m 5s), Eval_time (0m 1s)
4

 Epoch (4): Train_auc (0.5722634180085008), Valid_auc (0.5213149309912171) ,Training Average_loss (0.2650428790599108), Train_time (0m 5s), Eval_time (0m 1s)
5

 Epoch (5): Train_auc (0.5789798694885502), Valid_auc (0.528745294855709) ,T

Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForS


 Epoch (22): Train_auc (0.6638957807695538), Valid_auc (0.54364617314931) ,Training Average_loss (0.2655405956010024), Train_time (0m 5s), Eval_time (0m 1s)
BestValidAuc 0.614171 has a TestAuc of 0.544578 at epoch 1 
0

 Epoch (0): Train_auc (0.7682960438716796), Valid_auc (0.561676286072773) ,Training Average_loss (0.2701652484635512), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.5927380100214745) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.8918523809142999), Valid_auc (0.5999774153074027) ,Training Average_loss (0.25251959972083565), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6097434979718445) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.9238882135845466), Valid_auc (0.6044767879548306) ,Training Average_loss (0.21868330184370277), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6134460749224528) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train_auc (0.9594104992942626), Valid_auc (0.6214755332496863) ,Training Average_loss (0.16755947129180035), 

...


 Epoch (22): Train_auc (0.5732996825181033), Valid_auc (0.48895608531994983) ,Training Average_loss (0.2663395336518685), Train_time (0m 5s), Eval_time (0m 1s)
BestValidAuc 0.599390 has a TestAuc of 0.633188 at epoch 1 
0

 Epoch (0): Train_auc (0.7929439248599521), Valid_auc (0.6044818067754077) ,Training Average_loss (0.27189924828708173), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6511506800286329) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.8314479513620933), Valid_auc (0.5872170639899624) ,Training Average_loss (0.24913591692845027), Train_time (0m 5s), Eval_time (0m 1s)
2

 Epoch (2): Train_auc (0.562414281887138), Valid_auc (0.49971141781681305) ,Training Average_loss (0.25355972386896614), Train_time (0m 5s), Eval_time (0m 1s)
3

 Epoch (3): Train_auc (0.6897402666208171), Valid_auc (0.5565771643663739) ,Training Average_loss (0.2629288138821721), Train_time (0m 5s), Eval_time (0m 1s)
4

 Epoch (4): Train_auc (0.6415510929934064), Valid_auc (0.52582434127979

Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForS


 Epoch (21): Train_auc (0.5720885330656995), Valid_auc (0.5618670012547051) ,Training Average_loss (0.26692321021109816), Train_time (0m 5s), Eval_time (0m 1s)
BestValidAuc 0.604482 has a TestAuc of 0.651151 at epoch 0 
0

 Epoch (0): Train_auc (0.8037187666188177), Valid_auc (0.5659071518193225) ,Training Average_loss (0.2760312722374995), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6372709377236936) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.9426803229198738), Valid_auc (0.5957867001254704) ,Training Average_loss (0.24134745157013338), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6233017179670723) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.9730122394469256), Valid_auc (0.5986599749058971) ,Training Average_loss (0.16971625089645384), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.5752702219040802) , Test_eval_time (0m 0s) 
3

 Epoch (3): Train_auc (0.9861221125741223), Valid_auc (0.6034981179422836) ,Training Average_loss (0.1170559811250617

Some weights of the model checkpoint at /home/mai.kassem/Documents/Med-BERT/MedBERT Pretraining/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForS


 Epoch (27): Train_auc (0.9934609825145047), Valid_auc (0.5697666248431619) ,Training Average_loss (0.042441770768103494), Train_time (0m 5s), Eval_time (0m 1s)
BestValidAuc 0.623443 has a TestAuc of 0.581386 at epoch 6 
0

 Epoch (0): Train_auc (0.7791038233777035), Valid_auc (0.5661028858218319) ,Training Average_loss (0.27306947664668163), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6272888332140301) , Test_eval_time (0m 0s) 
1

 Epoch (1): Train_auc (0.9350285193907866), Valid_auc (0.5908205771643664) ,Training Average_loss (0.2413986290494601), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6196546170365068) , Test_eval_time (0m 0s) 
2

 Epoch (2): Train_auc (0.9663202788562655), Valid_auc (0.5839548306148056) ,Training Average_loss (0.18724807326992354), Train_time (0m 5s), Eval_time (0m 1s)
3

 Epoch (3): Train_auc (0.9779011207830689), Valid_auc (0.6019949811794227) ,Training Average_loss (0.14486427512019873), Train_time (0m 5s), Eval_time (0m 1s)

 Test_AUC (0.6

In [13]:
train_features = convert_EHRexamples_to_features(train_f, MAX_SEQ_LENGTH)
test_features = convert_EHRexamples_to_features(test_f, MAX_SEQ_LENGTH)
valid_features = convert_EHRexamples_to_features(valid_f, MAX_SEQ_LENGTH)
train = BERTdataEHR(train_features)
test = BERTdataEHR(test_features)
valid = BERTdataEHR(valid_features)

In [14]:
len(train_mbs[0][0][0])

100

In [15]:
df = pd.DataFrame(results)
df.columns = [
    "Model",
    "Run",
    "Train_size",
    "Test_size",
    "Valid_size",
    "Train_AUC",
    "Valid_AUC",
    "Test_AUC",
    "Best_Epoch",
]
df

,Model,Run,Train_size,Test_size,Valid_size,Train_AUC,Valid_AUC,Test_AUC,Best_Epoch
0,Bert only,0,12038,3439,1719,0.768260,0.614171,0.544578,1
1,Bert only,1,12038,3439,1719,0.959410,0.621476,0.596956,3
2,Bert only,2,12038,3439,1719,0.927739,0.640156,0.636481,1
3,Bert only,3,12038,3439,1719,0.977207,0.625817,0.579216,10
4,Bert only,4,12038,3439,1719,0.770143,0.581471,0.589424,0
5,Bert only,5,12038,3439,1719,0.950547,0.628203,0.609086,3
6,Bert only,6,12038,3439,1719,0.800577,0.599390,0.633188,1
7,Bert only,7,12038,3439,1719,0.792944,0.604482,0.651151,0
8,Bert only,8,12038,3439,1719,0.992838,0.623443,0.581386,6
9,Bert only,9,12038,3439,1719,0.993863,0.617508,0.564330,26


In [16]:
df.to_csv(
    root_path + "MedBERT_Results/results_gentacimin_antibiotic_2e3_finetuning.csv",
    index=False,
)